# The shared-house market in Europe

## Libraries and documents

In [5]:
import pandas as pd
import regex as re
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as px

In [6]:
bcn = pd.read_csv("../housekeeper/Data/bcn.csv", header=None)
madrid = pd.read_csv("../housekeeper/Data/madrid.csv", header=None)
london = pd.read_csv("../housekeeper/Data/london.csv", header=None)
berlin = pd.read_csv("../housekeeper/Data/berlin.csv", header=None)

## Data processing

In [7]:
def df_processor(df, name):
    """ Function that takes the df with the scraped information from each city and cleans it """
    
    df = df.drop_duplicates() #Dropping duplicated
    df.columns = ["title", "price", "details"] #Renaming columns

    df[["room", "flatmates"]] = df["details"].str.split("·", n=1, expand=True) # Splitting the details column

    #The flatmates column
    df["flatmates"] = df["flatmates"].str.rstrip(" COMPANYS DE PIS")#Deleting the str "companys de pis"

    #The room column
    df["room"] = df["room"].str.lower().str.strip() #Changing str as lowercase and removing whitespaces
    df = df.drop(columns="details") #Deleting the column

    #Filtering only "private room" & "shared room" as we are interested in this kind
    values = ["habitació compartida", "habitació privada"]
    df = df.loc[df['room'].isin(values)].copy() #Creating a copy of the df
    df["room"] = df["room"].replace("habitació privada", "private").replace("habitació compartida", "shared")

    #Dealing with the prices column
    df["price"] = df["price"].str.rstrip(" €/mes") #Deleting final string
    df["price"] = df["price"].str.rstrip(" £/mes")
    df["price"] = df["price"].str.rstrip(" $/mes")
    df["price"] = [re.sub(r'\d...','', str(x)) for x in df["price"]] #Deleting strikeout format string 
    df["price"] = df["price"].str.lstrip("€̶ ") #Deleting € symbols at the begining of str
    df["price"] = df["price"].str.lstrip("£̶ ") 
    df["price"] = df["price"].replace('-', np.nan) #Replacing - symbol to NaN value

    df = df.dropna() #Deleting rows with null values
    df[["price", "flatmates"]] = df[["price", "flatmates"]].apply(pd.to_numeric) #Changing price and nº flatmates to numeric
    df["flatmates"] = df["flatmates"].fillna(0) #Set nans to 0 for nº flatmates
    df["flatmates"] = df["flatmates"].astype(int)
    df["city"] = name # Create column with city name for concatenating df later
    
    return df

In [8]:
# Applying the function to the cities dfs
bcn = df_processor(bcn, "Barcelona")
madrid = df_processor(madrid, "Madrid")
london = df_processor(london, "London")
berlin = df_processor(berlin, "Berlin")

#Concatenating the datasets into one for analysis
datasets = [bcn, madrid, london, berlin]
df = pd.concat(datasets).reset_index(drop=True)
df["flatmates"] = df["flatmates"] + 1 #Summing a value to get total people sharing the house

#Removing outliers(prices with less than 100)
df = df[df["price"] > 100].copy()

from currency_converter import CurrencyConverter #Converting the prices in from GBP to EUR
c = CurrencyConverter()
condition = df["city"] == "London"
df["price"] = df['price'].where(~(condition), other=[c.convert(x, "GBP", "EUR") for x in df["price"]])

## Data analysis

### The dataset

With the information collected from Badi website and , the dataset contains a total of 13604 observations and 5 columns:
- `title`: contains the title of the flat
- `price`: contains the price in €
- `room`: indicates if the room is private or shared
- `flatmates`: is the number of flatmates
- `city`: indicates the city (Barcelona, Madrin, London or Berlin).

#### The rooms

![alt text](rooms2.png "Rooms")

The city with more flats on Badi's website is Barcelona, followed by Madrid, London and Berlin. This difference may be caused by the fact that the platform was launched in Germany less than a year ago.
Regarding the type of rooms, London stands out as the city with a major percentage of shared rooms agains private rooms with a 15%, while the other cities have a percentage of shared rooms from 1 to 10%.

#### The price variable

![alt text](prices.png "Prices")

The city with the highest average price for the room is London, with an average price of 746.5€. It can be confirmed that the prices in Spanish cities are lower.

#### The flatmates variable

![alt text](flatmates.png "Flatmates")

The graph below shows that 

In [10]:
%%HTML
<div class='tableauPlaceholder' id='viz1611080569975' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ba&#47;badi&#47;flatmates&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='badi&#47;flatmates' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ba&#47;badi&#47;flatmates&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1611080569975');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>